# Sistema de Recomendação - Introdução

Um sistema de recomendação é uma ferramenta tecnológica sofisticada, amplamente utilizada em plataformas online, que sugere produtos, serviços, informações ou opções a um usuário com base em uma variedade de fatores. Estes fatores incluem preferências pessoais do usuário, comportamento de navegação, histórico de compras, avaliações de outros usuários e uma série de outros dados relevantes.

Existem 3 principais tipos de Algoritmo de Recomendação:


#### Filtragem Colaborativa
A filtragem colaborativa é uma das abordagens mais populares para sistemas de recomendação. Ela se baseia no princípio de que usuários com preferências semelhantes em itens também têm probabilidades semelhantes de gostar de outros itens. O algoritmo utiliza dados históricos de interações entre usuários e itens (por exemplo, avaliações, compras, cliques) para identificar padrões de similaridade entre eles. Com base nessas similaridades, o sistema recomenda itens que outros usuários com gostos similares apreciaram.

Existem duas formas principais de filtragem colaborativa: a abordagem baseada em usuário, que recomenda itens com base nos interesses de usuários semelhantes, e a abordagem baseada em item, que recomenda itens com base em características semelhantes aos itens previamente apreciados pelo usuário.


#### Filtragem baseada em conteúdo
A filtragem baseada em conteúdo utiliza características e atributos dos itens para fazer recomendações. O algoritmo analisa informações descritivas sobre os itens e as combina com o perfil de preferências do usuário. Por exemplo, em um sistema de recomendação de filmes, o algoritmo pode considerar gênero, diretor, elenco e sinopse para sugerir filmes que possam agradar ao usuário com base em filmes que ele já assistiu e gostou.

Essa abordagem tende a ser mais personalizada, pois se concentra nas preferências específicas do usuário em vez de depender exclusivamente das opiniões de outros usuários.


#### Sistemas Híbridos
Os sistemas híbridos combinam as abordagens de filtragem colaborativa e filtragem baseada em conteúdo para melhorar a qualidade das recomendações. A ideia é que ao combinar as duas técnicas, é possível mitigar as limitações de cada uma individualmente.

Os algoritmos híbridos podem ser desenvolvidos de várias maneiras, como: utilizando filtragem colaborativa para inicialmente fazer recomendações amplas com base em tendências populares e, em seguida, refinando essas sugestões com a filtragem baseada em conteúdo para personalizá-las de acordo com as preferências do usuário. Outra abordagem comum é atribuir pesos diferentes às duas técnicas, dependendo do contexto da recomendação.

---

Vamos criar um modelo simples (colaborativo baseado em usuários) para compreender o conceito

In [66]:
from scipy import stats
import numpy as np

## Criação dos dados

In [67]:
matrix = np.array(
    [
    [8, 0, 3, 4, 6],
    [5, 6, 1, 8, 9],
    [8, 0, 0, 5, 10]
    ]
)

novoUser = [8, 0, 2, 3, 0]
naoAssistidos = [0, 1, 0, 0, 1]

nomeFilmes = ['Round 6', 'A Invocação do Mal', '9 Desconhecidos', 'You', 'La Casa de Papel']

Vamos calcular a similaridade entre os usuários com base nas notas dadas

In [68]:
#Para cada usuário, calcula a similaridade com o novo usuário
similarity = [0] * 3

for i in range(3):
    user = matrix[i,:]
    print(f"\nUser: {i}")
    print(f"User Note: {user}")

    # Notas dos usuários para filmes que foram assistidos pelo novo usuário (Nota != 0)
    UserNotZero = [nota_user for nota_novoUser, nota_user in zip(novoUser, user) if nota_novoUser != 0]
    NovoUserNotZero = [nota_novoUser for nota_novoUser in novoUser if nota_novoUser != 0]

    print(f"UserNotZero: {UserNotZero}")
    print(f"NovoUserNotZero: {NovoUserNotZero}")

    #Calcula a similaridade entre o novo usuário e o usuário atual
    similarity_users = stats.pearsonr(UserNotZero, NovoUserNotZero)
    similarity[i] = similarity_users

    print(f"Similarity User {i}: {similarity_users}")
    similarity[i] = similarity_users[0]

print(f"\n\nSimilarity with Users: {similarity}")



User: 0
User Note: [8 0 3 4 6]
UserNotZero: [8, 3, 4]
NovoUserNotZero: [8, 2, 3]
Similarity User 0: PearsonRResult(statistic=0.9994237971287664, pvalue=0.021612434272262178)

User: 1
User Note: [5 6 1 8 9]
UserNotZero: [5, 1, 8]
NovoUserNotZero: [8, 2, 3]
Similarity User 1: PearsonRResult(statistic=0.23621543814299703, pvalue=0.8481857831269412)

User: 2
User Note: [ 8  0  0  5 10]
UserNotZero: [8, 0, 5]
NovoUserNotZero: [8, 2, 3]
Similarity User 2: PearsonRResult(statistic=0.8723686098443353, pvalue=0.3251659459679011)


Similarity with Users: [0.9994237971287664, 0.23621543814299703, 0.8723686098443353]


Agora vamos calcular o peso das notas de cada usuário com base na similaridade deles com o novo usuário e fazer a soma acumulada do resultado

In [69]:
pesoNotas = np.zeros((3, 5))

for nUser in range(3):
    for nFilme in range(5):
        pesoNotas[nUser][nFilme] = naoAssistidos[nFilme] * matrix[nUser][nFilme] * similarity[nUser]

print(f"Peso das notas: \n{pesoNotas}")

Peso das notas: 
[[0.         0.         0.         0.         5.99654278]
 [0.         1.41729263 0.         0.         2.12593894]
 [0.         0.         0.         0.         8.7236861 ]]


In [70]:
accNotas = np.sum(pesoNotas.T, axis=1)
print(f"Notas Acumuladas: {accNotas}")

Notas Acumuladas: [ 0.          1.41729263  0.          0.         16.84616782]


Vamos fazer a soma acumulada dos valores de similaridade para cada filme

In [71]:
tempPeso = pesoNotas
tempPeso[pesoNotas > 0] = 1

tempSimilarity = np.zeros((3, 5))
for nUser in range(3):
    for nFilme in range(5):
        tempSimilarity[nUser][nFilme] = tempPeso[nUser][nFilme] * similarity[nUser]

print(f"Similarity: \n{tempSimilarity}")

accSimilarity = np.sum(tempSimilarity.T, axis=1)
print(f"Similarity Acumuladas: {accSimilarity}")

Similarity: 
[[0.         0.         0.         0.         0.9994238 ]
 [0.         0.23621544 0.         0.         0.23621544]
 [0.         0.         0.         0.         0.87236861]]
Similarity Acumuladas: [0.         0.23621544 0.         0.         2.10800785]


Por fim vamos normalizar os resultados para cada filme. Assim temos qual a provável nota que o novo Usuário daria para cada filme não assistido com base na similaridade com os outros usuários e as notas dadas por eles

In [72]:
notaFinal = [0] * 5
for nFilme in range(5):
    if accSimilarity[nFilme] > 0:
        notaFinal[nFilme] = accNotas[nFilme] / accSimilarity[nFilme]
    else:
        notaFinal[nFilme] = 0

print(f"Nota Final: {notaFinal}")

Nota Final: [0, 6.0, 0, 0, 7.991510972567145]


In [73]:
NnaoAssistidos = np.sum(naoAssistidos)

notasOrdenadasIndex = sorted(range(len(notaFinal)), key=lambda k: notaFinal[k], reverse=True)[:NnaoAssistidos]
for i in notasOrdenadasIndex:
    print(f"Filme: {nomeFilmes[i]}, Nota: {round(notaFinal[i],1)}")

Filme: La Casa de Papel, Nota: 8.0
Filme: A Invocação do Mal, Nota: 6.0
